In [14]:
from dotenv import load_dotenv
import os
import time
import fitz
import tiktoken
import openai
import textwrap
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeEmbeddings, PineconeVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain import hub

load_dotenv('C:/apis/.env') # path to your dotenv file
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
def mask_token(token, unmasked_chars=4):
    return token[:unmasked_chars] + '*' * (len(token) - unmasked_chars*2) + token[-unmasked_chars:]
print(mask_token(pinecone_api_key,4))
print(mask_token(openai_api_key,4))

pcsk*******************************************************************FZVA
sk-p************************************************************************************************************************************************************_5sA


### Loading PDF files

In [2]:
class PDFLoader:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path

    def extract_text(self):
        doc = fitz.open(self.pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text

if __name__ == '__main__':
    loader = PDFLoader('pdf/NVDA-form-10k.pdf')
    text = loader.extract_text()
    print(text)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
____________________________________________________________________________________________
FORM 10-K
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
    For the fiscal year ended January 28, 2024
OR
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
Commission file number: 0-23985
 
 
NVIDIA CORPORATION
(Exact name of registrant as specified in its charter)
Delaware
94-3177549
(State or other jurisdiction of
(I.R.S. Employer
incorporation or organization)
Identification No.)
   2788 San Tomas Expressway, Santa Clara, California
95051
  (Address of principal executive offices)
(Zip Code)
Registrant’s telephone number, including area code: (408) 486-2000
Securities registered pursuant to Section 12(b) of the Act:
Title of each class
Trading Symbol(s)
Name of each exchange on which registered
Common Stock, $0.001 par val

### Splitting text into Pinecone documents

In [3]:
def chunk_text_by_tokens(text, chunk_size, encoding_name="cl100k_base"):
    """
    Splits the text into chunks based on the number of tokens.
    """
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    return [encoding.decode(tokens[i:i + chunk_size]) for i in range(0, len(tokens), chunk_size)]

def process_text(text, chunk_size=1000):
    """
    Splits the text it into chunks, and generates the embeddings.
    """
    chunks = chunk_text_by_tokens(text, chunk_size)
    return chunks

chunks = process_text(text, chunk_size=800)

In [5]:
# Convert chunks list to Document objects with metadata
docs = [Document(page_content=text, metadata={"chunk": i}) for i, text in enumerate(chunks)]
display(docs)

[Document(metadata={'chunk': 0}, page_content='Table of Contents\nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C.\xa020549\n____________________________________________________________________________________________\nFORM 10-K\n☒\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\xa0\xa0\xa0\xa0For the fiscal year ended January\xa028, 2024\nOR\n☐\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nCommission file number: 0-23985\n\xa0\n\xa0\nNVIDIA CORPORATION\n(Exact name of registrant as specified in its charter)\nDelaware\n94-3177549\n(State or other jurisdiction of\n(I.R.S. Employer\nincorporation or organization)\nIdentification No.)\n\xa0\xa0\xa02788 San Tomas Expressway, Santa Clara, California\n95051\n\xa0\xa0(Address of principal executive offices)\n(Zip Code)\nRegistrant’s telephone number, including area code: (408) 486-2000\nSecurities registered pursuant to Section 12(b) of the

### Generating embeddings with Pinecone

In [6]:
model_name = 'multilingual-e5-large'
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=pinecone_api_key
)

In [7]:
pc = Pinecone(api_key=pinecone_api_key)

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)

index_name = "form-10k-docs"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embeddings.dimension,
        metric="cosine",
        spec=spec
    )
    # Wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# See that it is empty
pc.Index(index_name).describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'nvda-form-10k': {'vector_count': 97}},
 'total_vector_count': 97}

### Defining the Pinecone vector store

In [8]:
namespace = "nvda-form-10k"

docsearch = PineconeVectorStore.from_documents(
    documents=docs,
    index_name=index_name,
    embedding=embeddings,
    namespace=namespace
)

time.sleep(5)
pc.Index(index_name).describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'nvda-form-10k': {'vector_count': 97}},
 'total_vector_count': 97}

Direct search on vector space by index:

In [10]:
index = pc.Index(index_name)

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[33],
        namespace=namespace,
        top_k=1,
        include_values=False,
        include_metadata=True
    )
    print(query)

{'matches': [{'id': '32a156af-1944-459e-bfc0-ab463dcff1c4',
              'metadata': {'chunk': 28.0,
                           'text': ' and customers.\n'
                                   'For example, when integrating acquisition '
                                   'target systems into our own, we have '
                                   'experienced and may continue to experience '
                                   'challenges including lengthy and\n'
                                   'costly systems integration, delays in '
                                   'purchasing and shipping products, '
                                   'difficulties with system integration via '
                                   'electronic data interchange and other '
                                   'processes\n'
                                   'with our key suppliers and customers, and '
                                   'training and change management needs of '
                         

### Using the chatbot

In [11]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
retriever=docsearch.as_retriever()

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-4o-mini',
    temperature=0.0
)

combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

c:\Users\david\Documents\git\langchain-and-rag-notebooks\venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Question without context:

In [33]:
query1 = "Nvidia net income in 2024"
answer1_without_knowledge = llm.invoke(query1)
wrapper = textwrap.TextWrapper(width=100)
print("Query:\n", wrapper.fill(query1))
print("Answer without knowledge:\n", wrapper.fill(answer1_without_knowledge.content))

Query:
 Nvidia net income in 2024
Answer without knowledge:
 I'm sorry, but I don't have access to real-time financial data or projections for specific companies
like Nvidia beyond my last update in October 2023. For the most accurate and up-to-date information
on Nvidia's net income for 2024, I recommend checking financial news websites, Nvidia's official
investor relations page, or financial databases.


Question with RAG context:

In [34]:
answer1_with_knowledge = retrieval_chain.invoke({"input": query1})
print("Query 1:", wrapper.fill(query1))
print("Answer with knowledge:\n", wrapper.fill(answer1_with_knowledge['answer']))
print("\n")
print("Context used:")
print(answer1_with_knowledge['context'][0].id)
print(answer1_with_knowledge['context'][0].metadata)
print(answer1_with_knowledge['context'][0].page_content)

Query 1: Nvidia net income in 2024
Answer with knowledge:
 NVIDIA's net income for the year ended January 28, 2024, was $29,760 million.


Context used:
04b1781e-44cd-4baf-a884-ad556281a01f
{'chunk': 84.0}
Operating income (loss)
$
32,016 
$
5,846 
$
(4,890)
$
32,972 
Year Ended Jan 29, 2023:
 
 
 
Revenue
$
15,068 
$
11,906 
$
— 
$
26,974 
Operating income (loss)
$
5,083 
$
4,552 
$
(5,411)
$
4,224 
Year Ended Jan 30, 2022:
 
 
 
Revenue
$
11,046 
$
15,868 
$
— 
$
26,914 
Operating income (loss)
$
4,598 
$
8,492 
$
(3,049)
$
10,041 
78
Table of Contents
NVIDIA Corporation and Subsidiaries
Notes to the Consolidated Financial Statements
(Continued)
Year Ended
Jan 28, 2024
Jan 29, 2023
Jan 30, 2022
(In millions)
Reconciling items included in "All Other" category:
Stock-based compensation expense
$
(3,549)
$
(2,710)
$
(2,004)
Unallocated cost of revenue and operating expenses
(728)
(595)
(399)
Acquisition-related and other costs
(583)
(674)
(636)
IP-related and legal settlement costs
(40)

In [19]:
for item in answer1_with_knowledge['context']:
    print(item)

page_content='aries
Notes to the Consolidated Financial Statements
(Continued)
A reconciliation of gross unrecognized tax benefits is as follows:
 
Jan 28, 2024
Jan 29, 2023
Jan 30, 2022
 
(In millions)
Balance at beginning of period
$
1,238 
$
1,013 
$
776 
Increases in tax positions for current year
616 
268 
246 
Increases in tax positions for prior years
87 
1 
14 
Decreases in tax positions for prior years
(148)
(15)
(4)
Settlements
(104)
(9)
(8)
Lapse in statute of limitations
(19)
(20)
(11)
Balance at end of period
$
1,670 
$
1,238 
$
1,013 
Included in the balance of unrecognized tax benefits as of January 28, 2024 are $1.0 billion of tax benefits that would affect our effective tax rate if recognized.
We classify an unrecognized tax benefit as a current liability, or amount refundable, to the extent that we anticipate payment or receipt of cash for income taxes
within one year. The amount is classified as a long-term liability, or reduction of long-term amount refundable, if w

________________________________________________________________

_________________